## To combine the VFA orientation from 1st and 2nd cohort data

In [1]:
import glob as glob
import pandas as pd
import os
import shutil
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.stats import sem
from statsmodels.stats.anova import AnovaRM
import scikit_posthocs as sp
import statsmodels.formula.api as smf

In [2]:
import statsmodels.formula.api as smf
import scipy.stats as stats
import statsmodels.api as sm
import researchpy as rp

In [3]:
import pingouin as pg

In [4]:
disk_path="the parent folder directory" # put your parent folder directory into double quotes

In [5]:
coh1_male_path=disk_path+"coh1 male temp_2 folder directory"# put your temp2 subfolder directory into double quotes
coh1_female_path=disk_path+"coh1 female temp_2 folder directory"# put your temp2 subfolder directory into double quotes
coh2_path=disk_path+"coh2 temp_2 folder directory"# put your temp2 subfolder directory into double quotes

In [6]:
coh1_male_code=pd.read_csv(disk_path+"coh1 male frame info file directory", index_col=False) # put your frame info file subfolder directory into double quotes
coh1_female_code=pd.read_csv(disk_path+"coh1 female frame info file directory", index_col=False)# put your frame info file subfolder directory into double quotes
coh2_tot_code=pd.read_csv(disk_path+"coh2 frame info file directory", index_col=False)# put your frame info file subfolder directory into double quotes

### grasp before event LH time window orientation info

In [8]:
path_list=[coh1_male_path, coh1_female_path, coh2_path]
coh=["coh1", "coh1", "coh2"]
delte_str=[4,12,12]
code_file_name=[coh1_male_code["file_name"].tolist(), coh1_female_code["file_name"].tolist(), coh2_tot_code["vfa_csv_2"].tolist()]
nfps=[coh1_male_code["n_fps"].tolist(), coh1_female_code["n_fps"].tolist(), coh2_tot_code["n_fps"].tolist()]

event_list=["Hits - #", "Misses - #" , "Correct Rejections - #", "Mistakes - #" ]
bin_list=["15bin_1", "15bin_2","15bin_3","15bin_4","15bin_5","15bin_6"]

for i,m, h,j, d in zip(path_list,code_file_name, nfps, coh, delte_str) :
    os.chdir(i)
    
    save_path=i[:-7]+"\\ori_LH"
    if os.path.isdir(save_path) == False:
        os.mkdir(save_path)
    for n, b in zip(m, h):    
        file=pd.read_csv(n, low_memory=False)
        
        df_=file[file["event"].isin(["Stim Onset - End ITI", "Hits - #", "Misses - #" , "Correct Rejections - #", "Mistakes - #" ])].copy()

        hit_index=[]
        miss_index=[]
        mistake_index=[]
        corr_reject_index=[]
        for e in range(len(df_)-1):
            if df_.iloc[e]["event"]=="Stim Onset - End ITI" and df_.iloc[e+1]["event"]=="Hits - #":
                hit_index.append(df_.iloc[[e,e+1]]["sec_cal"].to_list())
            if df_.iloc[e]["event"]=="Stim Onset - End ITI" and df_.iloc[e+1]["event"]=="Misses - #":
                miss_index.append(df_.iloc[[e,e+1]]["sec_cal"].to_list())
            if df_.iloc[e]["event"]=="Stim Onset - End ITI" and df_.iloc[e+1]["event"]=="Mistakes - #":
                mistake_index.append(df_.iloc[[e,e+1]]["sec_cal"].to_list())
            if df_.iloc[e]["event"]=="Stim Onset - End ITI" and df_.iloc[e+1]["event"]=="Correct Rejections - #":
                corr_reject_index.append(df_.iloc[[e,e+1]]["sec_cal"].to_list())

        hit=[]
        miss=[]
        mistake=[]
        corr_reject=[]        
        for l, c in zip([hit_index,miss_index,mistake_index,corr_reject_index],
                        ["hit", "miss", "mistake", "corr_reject"]):
            for f, a in zip(l, range(1, len(l)+1)):
                b=file[(file["sec_cal"] >= f[0]) & (file["sec_cal"] <= f[1])].copy()
                b["trial_number"]=a
                b["event_type"]=c
                
                if c=="hit":
                    hit.append(b)            
                elif c=="miss":
                    miss.append(b)
                elif c=="mistake":
                    mistake.append(b)
                else:
                    corr_reject.append(b)            
        hit=pd.concat(hit, ignore_index=True)
        miss=pd.concat(miss, ignore_index=True)
        mistake=pd.concat(mistake, ignore_index=True)
        corr_reject=pd.concat(corr_reject, ignore_index=True)
        
        #add each bin's trial number into df according to event type
        event_list=["Hits - #", "Misses - #" ,"Mistakes - #", "Correct Rejections - #"]
        df=[hit, miss, mistake, corr_reject]
        for x, y in zip(event_list, df): 
            bin_n=[]
            bin_condition=[(y["15min_bin"]=="15bin_1"), (y["15min_bin"]=="15bin_2"),
                       (y["15min_bin"]=="15bin_3"),(y["15min_bin"]=="15bin_4"),
                       (y["15min_bin"]=="15bin_5"),(y["15min_bin"]=="15bin_6")]
            for z in bin_condition:
                bin_n.append(len(y[(y["event"]==x) & z]))
            y["bin_trial_n"]=np.select(bin_condition, bin_n)
        #combine four different event into individual file
        df=pd.concat([hit, miss, mistake, corr_reject], ignore_index=True)
        df.to_csv(save_path + "\\%s"%(n), index=False)

### calculate ori index
##### ori_index: determine LH trial orientation by using frame orientation, as long as one frame in LH is orientated, we determine this event as orientated frame, if not, we determine the event as blind. the ori_index=orientated trial number/ total trial number. 

#####  total orient_index= total orientated trial number/(hit trial number + miss trial number + mistake trial number + correct rejection trial number)
##### ex. event orient_index= total orientated trial number of miss/ total miss trial number

## for calculating ori index for TOT
For calculating split TOT event ori index, use orientation index event=mean(event orientation trial of TOT_1/total event trial of TOT_1 + event orientation trial of TOT_2/ total event trial of TOT_2)  

For calculating total TOT ori index use  orientation index total=(orited trial of hit + orited trial of miss + orited trail of mistake + orited trial of correct_reject)/(total trial of hit + total trial of miss + total trial of mistake + total trial of correct_reject)

In [70]:
#calculate orientaion index for individual sesson of individual mouse
path_list=[coh1_male_path, coh1_female_path, coh2_path]
coh=["coh1", "coh1", "coh2"]
delte_str=[4,12,12]
code_file_name=[coh1_male_code["file_name"].tolist(), coh1_female_code["file_name"].tolist(), coh2_tot_code["vfa_csv_2"].tolist()]
nfps=[coh1_male_code["n_fps"].tolist(), coh1_female_code["n_fps"].tolist(), coh2_tot_code["n_fps"].tolist()]
event_list=["hit", "miss", "mistake", "corr_reject"]

for i,m, h, j, d in zip(path_list,code_file_name, nfps, coh, delte_str) :
    os.chdir(i[:-7]+"\\ori_LH")
    
    save_path=i[:-7]+"\\15_ori_index_LH"
    if os.path.isdir(save_path) == False:
        os.mkdir(save_path)
        
    save_total=i[:-7]+"\\15_ori_index_LH\\total"
    if os.path.isdir(save_total) == False:
        os.mkdir(save_total)    
    
    for n, b in zip(m, h):
        df=pd.read_csv(n)
        #for c in ["hit", "miss", "mistake", "corr_reject"]:
        df=df.groupby(by=["15min_bin", "event_type", "trial_number", "blind_or_not"], as_index=False).count()
        df["cohort"]=j
        df["dt_id"]=n[:-d]
        df["animal_id"]=df["dt_id"][1][11:]
        df["id_cohort"]=df["animal_id"]+ "-" + df["cohort"]
        df["count_"]=1
        #display(df)
        df_=pd.pivot_table(df, index=["15min_bin", "event_type", "trial_number", "dt_id", "animal_id", "id_cohort"], 
                          columns=["blind_or_not"], values=["count_"]).reset_index().copy()#, aggfunc="count"
        
        df_.columns=[''.join(col) for col in df_.columns.values]
        df_["count_blind"] = df_["count_blind"].fillna(0)
        df_["count_oriented"] = df_["count_oriented"].fillna(0)
        #display(df_)
        
        ##### in column "blind_minus_ori_diff", if value ==  -1.0, meaning that trial is blind trial: if value=0 or 1, meaning that trial is oriented trial. 
        ##### the orientation index is calculated by oriented trial divided by trial number in each bin. 
        df_["blind_minus_ori_diff"]=df_["count_oriented"]-df_["count_blind"]         
        df_["bin_trial_n"]=0
        for x in event_list:    
            bin_condition=[(df_["15min_bin"]=="15bin_1") & (df_["event_type"]==x), (df_["15min_bin"]=="15bin_2")  & (df_["event_type"]==x),
                       (df_["15min_bin"]=="15bin_3") & (df_["event_type"]==x), (df_["15min_bin"]=="15bin_4") & (df_["event_type"]==x),
                       (df_["15min_bin"]=="15bin_5") & (df_["event_type"]==x), (df_["15min_bin"]=="15bin_6") & (df_["event_type"]==x)]
            bin_n=[]            
            for z in bin_condition:
                bin_n.append(len(df_[z]))
            df_["bin_trial_n"]=np.select(bin_condition, bin_n, default=df_["bin_trial_n"])            
        df_["onetozero"]=df_["blind_minus_ori_diff"].replace(1, 0) 
        #so in the "bind minus ori diff"col, only has value -1 or 0, -1 means blind, 0 means ori
        #display(df_)
        
        df_event=df_.groupby(by=["15min_bin", "event_type", "dt_id", "animal_id", "id_cohort"], as_index=False).agg(
            only_blind_trial_sum=('onetozero', 'sum'), bin_trial_n_sum=('bin_trial_n', 'mean'))        
        df_event["ori_trial"]=df_event["bin_trial_n_sum"]+df_event["only_blind_trial_sum"]
        #here, the ori index for this individual session is calculated.
        df_event["ori_index_event"]=df_event["ori_trial"]/df_event["bin_trial_n_sum"] 
        display(df_event)
        df_event.to_csv(save_path + "\\%s"%(n), index=False)
 

        #display(df_)
        df_total=df_event.groupby(by=["15min_bin", "dt_id", "animal_id", "id_cohort"], as_index=False).sum()
        df_total['ori_index_total_algorithm2']=df_total['ori_trial']/df_total['bin_trial_n_sum']
        df_total.drop(['ori_index_event'], axis=1, inplace=True)
        # ori_index_total_algoritum2 calculated individual session‘ total ori index first
        
        #df_total=df_.groupby(by=["15min_bin", "dt_id", "animal_id", "id_cohort"], as_index=False).agg(
            #only_blind_trial_sum=('onetozero', 'sum'), bin_trial_n_sum=('bin_trial_n', 'mean'))  #bin_trial_n_sum=('bin_trial_n', 'mean')
        #df_total["ori_trial"]=df_total["bin_trial_n_sum"]+df_total["only_blind_trial_sum"]
        
    #the above code, bin_trial_n_sum=('bin_trial_n', 'mean'), here the mean cause the error. 
    #here, the total ori index of individual session is calculated.
        #df_total["ori_index_total"]=df_total["ori_trial"]/df_total["bin_trial_n_sum"] 
        df_total.to_csv(save_total+"\\%s"%(n), index=False)
        display(df_total)

,15min_bin,event_type,dt_id,animal_id,id_cohort,only_blind_trial_sum,bin_trial_n_sum,ori_trial,ori_index_event
0,15bin_1,corr_reject,12-17-2021-X-N-5,X-N-5,X-N-5-coh1,-2.0,59.0,57.0,0.966102
1,15bin_1,hit,12-17-2021-X-N-5,X-N-5,X-N-5-coh1,0.0,41.0,41.0,1.000000
2,15bin_1,miss,12-17-2021-X-N-5,X-N-5,X-N-5-coh1,-4.0,25.0,21.0,0.840000
3,15bin_1,mistake,12-17-2021-X-N-5,X-N-5,X-N-5-coh1,0.0,7.0,7.0,1.000000
4,15bin_2,corr_reject,12-17-2021-X-N-5,X-N-5,X-N-5-coh1,-2.0,66.0,64.0,0.969697
5,15bin_2,hit,12-17-2021-X-N-5,X-N-5,X-N-5-coh1,0.0,18.0,18.0,1.000000
6,15bin_2,miss,12-17-2021-X-N-5,X-N-5,X-N-5-coh1,-3.0,52.0,49.0,0.942308
7,15bin_2,mistake,12-17-2021-X-N-5,X-N-5,X-N-5-coh1,-1.0,4.0,3.0,0.750000
8,15bin_3,corr_reject,12-17-2021-X-N-5,X-N-5,X-N-5-coh1,0.0,64.0,64.0,1.000000
9,15bin_3,hit,12-17-2021-X-N-5,X-N-5,X-N-5-coh1,0.0,15.0,15.0,1.000000


,15min_bin,dt_id,animal_id,id_cohort,event_type,only_blind_trial_sum,bin_trial_n_sum,ori_trial,ori_index_total_algorithm2
0,15bin_1,12-17-2021-X-N-5,X-N-5,X-N-5-coh1,corr_rejecthitmissmistake,-6.0,132.0,126.0,0.954545
1,15bin_2,12-17-2021-X-N-5,X-N-5,X-N-5-coh1,corr_rejecthitmissmistake,-6.0,140.0,134.0,0.957143
2,15bin_3,12-17-2021-X-N-5,X-N-5,X-N-5-coh1,corr_rejecthitmissmistake,-1.0,131.0,130.0,0.992366
3,15bin_4,12-17-2021-X-N-5,X-N-5,X-N-5-coh1,corr_rejecthitmissmistake,0.0,124.0,124.0,1.000000
4,15bin_5,12-17-2021-X-N-5,X-N-5,X-N-5-coh1,corr_rejecthitmissmistake,-3.0,143.0,140.0,0.979021
5,15bin_6,12-17-2021-X-N-5,X-N-5,X-N-5-coh1,corr_rejecthitmissmistake,-8.0,130.0,122.0,0.938462


,15min_bin,event_type,dt_id,animal_id,id_cohort,only_blind_trial_sum,bin_trial_n_sum,ori_trial,ori_index_event
0,15bin_1,corr_reject,12-17-2021-X-RL-6,X-RL-6,X-RL-6-coh1,-2.0,50.0,48.0,0.960000
1,15bin_1,hit,12-17-2021-X-RL-6,X-RL-6,X-RL-6-coh1,0.0,45.0,45.0,1.000000
2,15bin_1,miss,12-17-2021-X-RL-6,X-RL-6,X-RL-6-coh1,-1.0,16.0,15.0,0.937500
3,15bin_1,mistake,12-17-2021-X-RL-6,X-RL-6,X-RL-6-coh1,0.0,12.0,12.0,1.000000
4,15bin_2,corr_reject,12-17-2021-X-RL-6,X-RL-6,X-RL-6-coh1,-1.0,57.0,56.0,0.982456
5,15bin_2,hit,12-17-2021-X-RL-6,X-RL-6,X-RL-6-coh1,0.0,38.0,38.0,1.000000
6,15bin_2,miss,12-17-2021-X-RL-6,X-RL-6,X-RL-6-coh1,0.0,27.0,27.0,1.000000
7,15bin_2,mistake,12-17-2021-X-RL-6,X-RL-6,X-RL-6-coh1,0.0,7.0,7.0,1.000000
8,15bin_3,corr_reject,12-17-2021-X-RL-6,X-RL-6,X-RL-6-coh1,-4.0,69.0,65.0,0.942029
9,15bin_3,hit,12-17-2021-X-RL-6,X-RL-6,X-RL-6-coh1,0.0,9.0,9.0,1.000000


,15min_bin,dt_id,animal_id,id_cohort,event_type,only_blind_trial_sum,bin_trial_n_sum,ori_trial,ori_index_total_algorithm2
0,15bin_1,12-17-2021-X-RL-6,X-RL-6,X-RL-6-coh1,corr_rejecthitmissmistake,-3.0,123.0,120.0,0.975610
1,15bin_2,12-17-2021-X-RL-6,X-RL-6,X-RL-6-coh1,corr_rejecthitmissmistake,-1.0,129.0,128.0,0.992248
2,15bin_3,12-17-2021-X-RL-6,X-RL-6,X-RL-6-coh1,corr_rejecthitmissmistake,-6.0,142.0,136.0,0.957746
3,15bin_4,12-17-2021-X-RL-6,X-RL-6,X-RL-6-coh1,corr_rejecthitmissmistake,-9.0,125.0,116.0,0.928000
4,15bin_5,12-17-2021-X-RL-6,X-RL-6,X-RL-6-coh1,corr_rejecthitmissmistake,-1.0,140.0,139.0,0.992857
5,15bin_6,12-17-2021-X-RL-6,X-RL-6,X-RL-6-coh1,corr_rejecthitmissmistake,-6.0,134.0,128.0,0.955224


,15min_bin,event_type,dt_id,animal_id,id_cohort,only_blind_trial_sum,bin_trial_n_sum,ori_trial,ori_index_event
0,15bin_1,corr_reject,12-17-2021-X-R-8,X-R-8,X-R-8-coh1,-1.0,62.0,61.0,0.983871
1,15bin_1,hit,12-17-2021-X-R-8,X-R-8,X-R-8-coh1,0.0,49.0,49.0,1.000000
2,15bin_1,miss,12-17-2021-X-R-8,X-R-8,X-R-8-coh1,-1.0,18.0,17.0,0.944444
3,15bin_1,mistake,12-17-2021-X-R-8,X-R-8,X-R-8-coh1,0.0,5.0,5.0,1.000000
4,15bin_2,corr_reject,12-17-2021-X-R-8,X-R-8,X-R-8-coh1,-4.0,68.0,64.0,0.941176
5,15bin_2,hit,12-17-2021-X-R-8,X-R-8,X-R-8-coh1,0.0,28.0,28.0,1.000000
6,15bin_2,miss,12-17-2021-X-R-8,X-R-8,X-R-8-coh1,-7.0,42.0,35.0,0.833333
7,15bin_2,mistake,12-17-2021-X-R-8,X-R-8,X-R-8-coh1,0.0,2.0,2.0,1.000000
8,15bin_3,corr_reject,12-17-2021-X-R-8,X-R-8,X-R-8-coh1,-3.0,69.0,66.0,0.956522
9,15bin_3,hit,12-17-2021-X-R-8,X-R-8,X-R-8-coh1,0.0,12.0,12.0,1.000000


...

,15min_bin,dt_id,animal_id,id_cohort,event_type,only_blind_trial_sum,bin_trial_n_sum,ori_trial,ori_index_total_algorithm2
0,15bin_1,03-31-2022-X-N-5,X-N-5,X-N-5-coh2,corr_rejecthitmiss,-4.0,138.0,134.0,0.971014
1,15bin_2,03-31-2022-X-N-5,X-N-5,X-N-5-coh2,corr_rejecthitmissmistake,-3.0,148.0,145.0,0.979730
2,15bin_3,03-31-2022-X-N-5,X-N-5,X-N-5-coh2,corr_rejecthitmissmistake,-10.0,146.0,136.0,0.931507
3,15bin_4,03-31-2022-X-N-5,X-N-5,X-N-5-coh2,corr_rejecthitmissmistake,-12.0,147.0,135.0,0.918367
4,15bin_5,03-31-2022-X-N-5,X-N-5,X-N-5-coh2,corr_rejectmiss,-11.0,149.0,138.0,0.926174
5,15bin_6,03-31-2022-X-N-5,X-N-5,X-N-5-coh2,corr_rejecthitmissmistake,-18.0,146.0,128.0,0.876712


,15min_bin,event_type,dt_id,animal_id,id_cohort,only_blind_trial_sum,bin_trial_n_sum,ori_trial,ori_index_event
0,15bin_1,corr_reject,03-31-2022-Z-N-13,Z-N-13,Z-N-13-coh2,-2.0,59.0,57.0,0.966102
1,15bin_1,hit,03-31-2022-Z-N-13,Z-N-13,Z-N-13-coh2,0.0,35.0,35.0,1.000000
2,15bin_1,miss,03-31-2022-Z-N-13,Z-N-13,Z-N-13-coh2,-2.0,30.0,28.0,0.933333
3,15bin_1,mistake,03-31-2022-Z-N-13,Z-N-13,Z-N-13-coh2,0.0,6.0,6.0,1.000000
4,15bin_2,corr_reject,03-31-2022-Z-N-13,Z-N-13,Z-N-13-coh2,-5.0,66.0,61.0,0.924242
5,15bin_2,hit,03-31-2022-Z-N-13,Z-N-13,Z-N-13-coh2,0.0,20.0,20.0,1.000000
6,15bin_2,miss,03-31-2022-Z-N-13,Z-N-13,Z-N-13-coh2,-5.0,47.0,42.0,0.893617
7,15bin_2,mistake,03-31-2022-Z-N-13,Z-N-13,Z-N-13-coh2,0.0,4.0,4.0,1.000000
8,15bin_3,corr_reject,03-31-2022-Z-N-13,Z-N-13,Z-N-13-coh2,-3.0,69.0,66.0,0.956522
9,15bin_3,hit,03-31-2022-Z-N-13,Z-N-13,Z-N-13-coh2,0.0,11.0,11.0,1.000000


,15min_bin,dt_id,animal_id,id_cohort,event_type,only_blind_trial_sum,bin_trial_n_sum,ori_trial,ori_index_total_algorithm2
0,15bin_1,03-31-2022-Z-N-13,Z-N-13,Z-N-13-coh2,corr_rejecthitmissmistake,-4.0,130.0,126.0,0.969231
1,15bin_2,03-31-2022-Z-N-13,Z-N-13,Z-N-13-coh2,corr_rejecthitmissmistake,-10.0,137.0,127.0,0.927007
2,15bin_3,03-31-2022-Z-N-13,Z-N-13,Z-N-13-coh2,corr_rejecthitmissmistake,-10.0,143.0,133.0,0.930070
3,15bin_4,03-31-2022-Z-N-13,Z-N-13,Z-N-13-coh2,corr_rejecthitmissmistake,-10.0,137.0,127.0,0.927007
4,15bin_5,03-31-2022-Z-N-13,Z-N-13,Z-N-13-coh2,corr_rejecthitmissmistake,-12.0,143.0,131.0,0.916084
5,15bin_6,03-31-2022-Z-N-13,Z-N-13,Z-N-13-coh2,corr_rejecthitmissmistake,-10.0,132.0,122.0,0.924242


,15min_bin,event_type,dt_id,animal_id,id_cohort,only_blind_trial_sum,bin_trial_n_sum,ori_trial,ori_index_event
0,15bin_1,corr_reject,04-05-2022-Z-R-16,Z-R-16,Z-R-16-coh2,0.0,56.0,56.0,1.000000
1,15bin_1,hit,04-05-2022-Z-R-16,Z-R-16,Z-R-16-coh2,0.0,39.0,39.0,1.000000
2,15bin_1,miss,04-05-2022-Z-R-16,Z-R-16,Z-R-16-coh2,-8.0,25.0,17.0,0.680000
3,15bin_1,mistake,04-05-2022-Z-R-16,Z-R-16,Z-R-16-coh2,0.0,9.0,9.0,1.000000
4,15bin_2,corr_reject,04-05-2022-Z-R-16,Z-R-16,Z-R-16-coh2,-3.0,70.0,67.0,0.957143
5,15bin_2,hit,04-05-2022-Z-R-16,Z-R-16,Z-R-16-coh2,0.0,24.0,24.0,1.000000
6,15bin_2,miss,04-05-2022-Z-R-16,Z-R-16,Z-R-16-coh2,-6.0,48.0,42.0,0.875000
7,15bin_2,mistake,04-05-2022-Z-R-16,Z-R-16,Z-R-16-coh2,0.0,1.0,1.0,1.000000
8,15bin_3,corr_reject,04-05-2022-Z-R-16,Z-R-16,Z-R-16-coh2,-8.0,67.0,59.0,0.880597
9,15bin_3,hit,04-05-2022-Z-R-16,Z-R-16,Z-R-16-coh2,0.0,34.0,34.0,1.000000


,15min_bin,dt_id,animal_id,id_cohort,event_type,only_blind_trial_sum,bin_trial_n_sum,ori_trial,ori_index_total_algorithm2
0,15bin_1,04-05-2022-Z-R-16,Z-R-16,Z-R-16-coh2,corr_rejecthitmissmistake,-8.0,129.0,121.0,0.937984
1,15bin_2,04-05-2022-Z-R-16,Z-R-16,Z-R-16-coh2,corr_rejecthitmissmistake,-9.0,143.0,134.0,0.937063
2,15bin_3,04-05-2022-Z-R-16,Z-R-16,Z-R-16-coh2,corr_rejecthitmissmistake,-9.0,140.0,131.0,0.935714
3,15bin_4,04-05-2022-Z-R-16,Z-R-16,Z-R-16-coh2,corr_rejecthitmissmistake,-4.0,139.0,135.0,0.971223
4,15bin_5,04-05-2022-Z-R-16,Z-R-16,Z-R-16-coh2,corr_rejecthitmissmistake,-8.0,143.0,135.0,0.944056
5,15bin_6,04-05-2022-Z-R-16,Z-R-16,Z-R-16-coh2,corr_rejecthitmissmistake,-9.0,143.0,134.0,0.937063


### plot ori index event

In [23]:
# orientation index=mean(orientation trial of TOT-1/total trial of TOT-1 + orientation trial of TOT-2/ total trial of TOT-2)
path_list=[coh1_male_path, coh1_female_path, coh2_path]
df_list=[]

for i in path_list:
    file_list=glob.glob(i[:-7]+"\\15_ori_index_LH\\*.csv")
    df_list=df_list+file_list
    
#combine 15min bin df from three folder

df=pd.DataFrame()
for i in df_list:
    df_=pd.read_csv(i)
    if len(df_)<8:
        print(i)
    df=pd.concat([df, df_], ignore_index=True)
    
df['group_id']=df['animal_id'].apply(lambda x:x[0])
df['gander']=np.where(df['group_id'].isin(["W", "X", "Y","Z"]), "male", "female")
display(df[df['id_cohort']=='W-L-2-coh1'])

#since mouse have 2 or 4 time TOT, do the groupby to calculate average the value of orient_index_event all the TOT test before plot
df_=df.drop(['dt_id'],axis=1).groupby(by=["15min_bin", "event_type", "animal_id", "group_id", "gander", "id_cohort"], as_index=False).mean()

display(df_[df_['id_cohort']=='W-L-2-coh1'])
#ori_index_event column is calculated by average orient index within TOT session
# the event_index is calculated by average trial n first, and then calculated the orientation index
#df['event_index']=df['ori_trial']/df['bin_trial_n_sum'] 

event=["hit", "miss", "mistake", "corr_reject"]
gander=["male", "female"]
color_list=['royalblue', 'coral']
for i, p in zip(gander, color_list) :
    for j in event:        
        title="%s_%s_ori_index_before_event_15min_bin"%(i, j)
        t=df_[(df_["gander"]==i) & (df_["event_type"]==j)].copy()
        t=t.rename(columns={"15min_bin": "min_15bin"})
        
        plot=sns.pointplot(data=t, x="min_15bin", y="ori_index_event", capsize=.01, errorbar=('ci', 68))
        plot.set(xlabel="")
        plot.set(ylabel="orientation index")
        plt.title(title)
        plt.ylim(0.75, 1.05)
        plt.xticks(rotation = 15, ha='right')
    #     h = plt.gca().get_lines()
    #     lg = plt.legend(handles=h, labels=['oriented', 'blind'], loc='best')
        plt.legend(bbox_to_anchor=(1.25, 1),borderaxespad=0)

        save_path=disk_path+"......" #replace ...... with the directory you want to save the plot to

        if os.path.isdir(save_path ) == False:
            os.mkdir(save_path)
        t.to_csv(save_path+"\\%s.csv"%(title), index=False)
        plt.savefig(save_path+"\\%s.png"%(title), dpi=800, bbox_inches="tight")
        plt.close()


,15min_bin,event_type,dt_id,animal_id,id_cohort,only_blind_trial_sum,bin_trial_n_sum,ori_trial,ori_index_event,group_id,gander
0,15bin_1,corr_reject,12-07-2021-W-L-2,W-L-2,W-L-2-coh1,-6.0,56.0,50.0,0.892857,W,male
1,15bin_1,hit,12-07-2021-W-L-2,W-L-2,W-L-2-coh1,0.0,47.0,47.0,1.000000,W,male
2,15bin_1,miss,12-07-2021-W-L-2,W-L-2,W-L-2-coh1,-5.0,18.0,13.0,0.722222,W,male
3,15bin_1,mistake,12-07-2021-W-L-2,W-L-2,W-L-2-coh1,0.0,9.0,9.0,1.000000,W,male
4,15bin_2,corr_reject,12-07-2021-W-L-2,W-L-2,W-L-2-coh1,-2.0,62.0,60.0,0.967742,W,male
5,15bin_2,hit,12-07-2021-W-L-2,W-L-2,W-L-2-coh1,0.0,49.0,49.0,1.000000,W,male
6,15bin_2,miss,12-07-2021-W-L-2,W-L-2,W-L-2-coh1,-5.0,21.0,16.0,0.761905,W,male
7,15bin_2,mistake,12-07-2021-W-L-2,W-L-2,W-L-2-coh1,0.0,5.0,5.0,1.000000,W,male
8,15bin_3,corr_reject,12-07-2021-W-L-2,W-L-2,W-L-2-coh1,-1.0,56.0,55.0,0.982143,W,male
9,15bin_3,hit,12-07-2021-W-L-2,W-L-2,W-L-2-coh1,0.0,43.0,43.0,1.000000,W,male


,15min_bin,event_type,animal_id,group_id,gander,id_cohort,only_blind_trial_sum,bin_trial_n_sum,ori_trial,ori_index_event
11,15bin_1,corr_reject,W-L-2,W,male,W-L-2-coh1,-4.0,57.0,53.0,0.929187
43,15bin_1,hit,W-L-2,W,male,W-L-2-coh1,0.0,50.5,50.5,1.000000
75,15bin_1,miss,W-L-2,W,male,W-L-2-coh1,-3.0,14.5,11.5,0.815657
107,15bin_1,mistake,W-L-2,W,male,W-L-2-coh1,0.0,8.0,8.0,1.000000
139,15bin_2,corr_reject,W-L-2,W,male,W-L-2-coh1,-2.5,63.5,61.0,0.960794
171,15bin_2,hit,W-L-2,W,male,W-L-2-coh1,0.0,47.5,47.5,1.000000
203,15bin_2,miss,W-L-2,W,male,W-L-2-coh1,-3.0,22.5,19.5,0.860119
235,15bin_2,mistake,W-L-2,W,male,W-L-2-coh1,0.0,4.0,4.0,1.000000
267,15bin_3,corr_reject,W-L-2,W,male,W-L-2-coh1,-4.0,54.5,50.5,0.925034
299,15bin_3,hit,W-L-2,W,male,W-L-2-coh1,0.0,37.0,37.0,1.000000


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that 

In [27]:
sns.set(style="ticks",rc={"lines.linewidth": 0.7})
#combine individual event into one plot ori_index
df=pd.DataFrame()
for i in df_list:
    df_=pd.read_csv(i)
    df=pd.concat([df, df_], ignore_index=True)
    
event=["hit", "miss", "mistake", "corr_reject"]
gander=["male", "female"]
color_list=['royalblue', 'coral']

for i, p in zip(gander, color_list) :   
        title="%s_ori_index_before_event_15min_bin"%i
        t=df[(df["gander"]==i)].copy()
        t=t.rename(columns={"15min_bin": "min_15bin"})

        plot=sns.pointplot(data=t, x="min_15bin", y="ori_index_event", hue="event_type",  hue_order=["hit", "miss", "mistake", "corr_reject"], palette=[p, p, p, p], 
                            markers=["o", "x","D", "*"], capsize=.01, errorbar=('ci', 68), linestyles=['-', '-.', ':', '--']) #,
        plt.setp(plot.collections, sizes=[70])
        plot.set(xlabel="")
        plot.set(ylabel="orientation index")
        plt.title(title)
        plt.xticks(rotation = 15, ha='right')
        plot.yaxis.set_ticks(np.arange(0.80, 1.05, 0.05))
    #     h = plt.gca().get_lines()
    #     lg = plt.legend(handles=h, labels=['oriented', 'blind'], loc='best')
        plt.legend(bbox_to_anchor=(1.30, 1),borderaxespad=0)

        save_path=disk_path+"......" #replace ...... with the directory you want to save the plot to

        if os.path.isdir(save_path ) == False:
            os.mkdir(save_path)
        #corrected means change the solid line to dot line based on different event    
        t.to_csv(save_path+"\\%s_corrected.csv"%(title), index=False)
        plt.savefig(save_path+"\\%s_corrected.png"%(title), dpi=800, bbox_inches="tight")
        plt.close()
        

C:\Users\ye.li\AppData\Local\anaconda3\Lib\site-packages\seaborn\categorical.py:1728: UserWarning: You passed a edgecolor/edgecolors ((0.2549019607843137, 0.4117647058823529, 0.8823529411764706)) for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  ax.scatter(x, y, label=hue_level,
C:\Users\ye.li\AppData\Local\anaconda3\Lib\site-packages\seaborn\categorical.py:1728: UserWarning: You passed a edgecolor/edgecolors ((0.2549019607843137, 0.4117647058823529, 0.8823529411764706)) for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  ax.scatter(x, y, label=hue_level,
C:\Users\ye.li\AppData\Local\anaconda3\Lib\site-packages\seaborn\categorical.py:1728: UserWarning: You passed a edgecolor/edgecolors ((1.0, 0.4980392156862745, 0.3137254901960784)) for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the

## plot ori index total

### separate gander and combine gander

In [74]:
sns.set(style="ticks",rc={"lines.linewidth": 0.7})

path_list=[coh1_male_path, coh1_female_path, coh2_path]
df_list=[]

for i in path_list:
    file_list=glob.glob(i[:-7]+"\\15_ori_index_LH\\total\\*.csv")
    df_list=df_list+file_list

save_path=disk_path+"\\CPT_TOT_manuscript\\data processing\\fig\\15_ori_index_LH_total"
if os.path.isdir(save_path ) == False:
    os.mkdir(save_path)    
#combine 15min bin df from three parts

df=pd.DataFrame()
for i in df_list:
    df_=pd.read_csv(i)
#     if len(df_)<8:
#         print(i)
    df=pd.concat([df, df_], ignore_index=True)
df['group_id']=df['animal_id'].apply(lambda x:x[0])
df['gander']=np.where(df['group_id'].isin(["W", "X", "Y","Z"]), "male", "female")
#display(df[df['id_cohort']=='W-L-2-coh1'])


df_=df.drop(['dt_id', 'animal_id', 'event_type', 'group_id'],axis=1).groupby(by=["15min_bin", "gander", "id_cohort"], as_index=False).mean()
#ori_index_total_algorithm2 column is calculated by average of each TOT session's orient index
#total_index_algorithm1 is calculated by average trial n from two TOT session first, and then calculated the orientation index
#the ori_index_total_algorithm2 method is used.

df_['total_index_algorithm1']=df_['ori_trial']/df_['bin_trial_n_sum'] # total_index is calculated by average trial number first
#display(df[df['id_cohort']=='W-L-2-coh1'])
#df=df.groupby(by=['15min_bin', "gander"])
#display(df[df['id_cohort']=='A-L-2-coh2'])

gander=["male", "female"]
color_list=['royalblue', 'coral']
df_=df_.rename(columns={"15min_bin": "min_15bin"})

for j in ['ori_index_total_algorithm2']:   #'total_index_algorithm1', 
    for i, p in zip(gander, color_list) :   
        title="%s_total_ori_index_in_LH_15min_bin"%i
        t=df_[(df_["gander"]==i)].copy()
        #t=t.rename(columns={"15min_bin": "min_15bin"})
        #display(t)
        plot=sns.pointplot(data=t, x="min_15bin", y=j, capsize=.01, errorbar=('ci', 68), color=p) #linestyles=['-', '-.', ':', '--'],
        plt.setp(plot.collections, sizes=[70])
        plot.set(xlabel="")
        plot.set(ylabel=j)
        plt.title(title)
        plt.xticks(rotation = 15, ha='right')
        plot.yaxis.set_ticks(np.arange(0.20, 1.05, 0.05))
    #     h = plt.gca().get_lines()
    #     lg = plt.legend(handles=h, labels=['oriented', 'blind'], loc='best')
        plt.legend(bbox_to_anchor=(1.30, 1),borderaxespad=0)

        t.to_csv(save_path+"\\%s_%s.csv"%(title, j), index=False)
        plt.savefig(save_path+"\\%s_%s.png"%(title, j), dpi=800, bbox_inches="tight")
        plt.close()

        plot=sns.pointplot(data=t, x="min_15bin", y=j, capsize=.01, errorbar=('ci', 68), color=p) #linestyles=['-', '-.', ':', '--'],
        plt.setp(plot.collections, sizes=[70])
        plot.set(xlabel="")
        plot.set(ylabel=j)
        plt.title(title)
        plt.xticks(rotation = 15, ha='right')
        plt.ylim(0.2, 1.20)
        plt.legend(bbox_to_anchor=(1.30, 1),borderaxespad=0)
        plt.savefig(save_path+"\\%s_%s_corrected_scale.png"%(title, j), dpi=800, bbox_inches="tight")
        plt.close()

    plot=sns.pointplot(data=df_, x="min_15bin", y=j,hue='gander', hue_order=['male', 'female'], palette=['royalblue', 'coral'], capsize=.01, errorbar=('ci', 68), color=p) #linestyles=['-', '-.', ':', '--'],
    plt.setp(plot.collections, sizes=[70])
    plot.set(xlabel="")
    plot.set(ylabel=j)
    title='male_vs_female_total_ori_index_in_LH'
    plt.title(title)
    plt.xticks(rotation = 15, ha='right')
    plot.yaxis.set_ticks(np.arange(0.80, 1.05, 0.05))
    plt.legend(bbox_to_anchor=(1.30, 1),borderaxespad=0)    
    save_path=disk_path+"\\CPT_TOT_manuscript\\data processing\\fig\\15_ori_index_LH_total"    
    plt.savefig(save_path+"\\%s_%s_corrected.png"%(title, j), dpi=800, bbox_inches="tight")
    plt.close()
    df_.to_csv(save_path+"\\%s_%s_corrected.csv"%(title,j), index=False)

    plot=sns.pointplot(data=df_, x="min_15bin", y=j,hue='gander', hue_order=['male', 'female'], palette=['royalblue', 'coral'], capsize=.01, errorbar=('ci', 68), color=p) #linestyles=['-', '-.', ':', '--'],
    plt.setp(plot.collections, sizes=[70])
    plot.set(xlabel="")
    plot.set(ylabel=j)
    title='male_vs_female_total_ori_index_in_LH'
    plt.title(title)
    plt.xticks(rotation = 15, ha='right')
    plt.ylim(0.2, 1.20)
    #     h = plt.gca().get_lines()
    #     lg = plt.legend(handles=h, labels=['oriented', 'blind'], loc='best')
    plt.legend(bbox_to_anchor=(1.30, 1),borderaxespad=0)
    save_path=disk_path+"\\CPT_TOT_manuscript\\data processing\\fig\\15_ori_index_LH_total"    
    plt.savefig(save_path+"\\%s_%s_corrected_scale.png"%(title,j), dpi=800, bbox_inches="tight")
    plt.close()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that 

## calculate response rate and passive rate： 
### response rate=($N_{Hit}$+$N_{Mistake}$) / ($N_{Hit}$+$N_{Mistake}$+$N_{Miss}$+$N_{Corr reject}$ )
### passive rate=($N_{Miss}$+$N_{Corr reject}$) / ($N_{Hit}$+$N_{Mistake}$+$N_{Miss}$+$N_{Corr reject}$ )

In [116]:
#calculate response rate and passive rate for individual sesson of individual mouse
path_list=[coh1_male_path, coh1_female_path, coh2_path]
coh=["coh1", "coh1", "coh2"]
delte_str=[4,12,12]
code_file_name=[coh1_male_code["file_name"].tolist(), coh1_female_code["file_name"].tolist(), coh2_tot_code["vfa_csv_2"].tolist()]
nfps=[coh1_male_code["n_fps"].tolist(), coh1_female_code["n_fps"].tolist(), coh2_tot_code["n_fps"].tolist()]
event_list=["Hits - #", "Misses - #", "Mistakes - #", "Correct Rejections - #"]

for i,m, h, j, d in zip(path_list,code_file_name, nfps, coh, delte_str) :
    os.chdir(i[:-7]+"\\ori_LH")
    
    save_path=i[:-7]+"\\15_response_rate_LH"
    if os.path.isdir(save_path) == False:
        os.mkdir(save_path)

    
    for n, b in zip(m, h):
        df=pd.read_csv(n)
        df_=df[df["event"].isin(event_list)][["event","15min_bin", "bin_trial_n"]].copy()
        
        t=df_.groupby(by=['event','15min_bin'], as_index=False).agg(
            event_count=('event', 'count'),
            bin_trial_n_sum=('bin_trial_n', 'mean'))
        
        t_=pd.pivot_table(t, index=["15min_bin"],columns=["event"], values=["event_count"]).reset_index()
        t_.columns=['_'.join(col) for col in t_.columns.values]
        
        t_["cohort"]=j
        t_["dt_id"]=n[:-d]
        t_["animal_id"]=t_["dt_id"][1][11:]
        t_['group_id']=t_['animal_id'].apply(lambda x:x[0])
        t_['gander']=np.where(t_['group_id'].isin(["W", "X", "Y","Z"]), "male", "female")
        t_["id_cohort"]=t_["animal_id"]+ "-" + t_["cohort"] 
        t_.rename(columns={'event_count_Correct Rejections - #': 'n_corrj', 'event_count_Hits - #': 'n_hit', 
                           'event_count_Misses - #': 'n_miss', 'event_count_Mistakes - #': 'n_mistake', "15min_bin_": "min_15bin"}, inplace=True)
        t_=t_.fillna(0)
        t_['response_rate']=(t_['n_hit'] + t_['n_mistake'])/(t_['n_hit'] + t_['n_mistake'] + t_['n_miss'] + t_['n_corrj'])
        t_['passive_rate']=(t_['n_miss'] + t_['n_corrj'])/(t_['n_hit'] + t_['n_mistake'] + t_['n_miss'] + t_['n_corrj'])
        t_.to_csv(save_path + "\\%s"%(n), index=False)

In [119]:
# merge response rate and total orientation index plot
sns.set(font_scale = 1.5)
sns.set_style(style="ticks")

path=disk_path+"\\CPT_TOT_manuscript\\data processing\\fig\\15_ori_index_LH_total\\15_ori_index_LH_total_adjust_scale"


#combine response rate df to one
path_list=[coh1_male_path, coh1_female_path, coh2_path]
df_list=[]

for i in path_list:
    file_list=glob.glob(i[:-7]+"\\15_response_rate_LH\\*.csv")
    df_list=df_list+file_list
    
rr=pd.DataFrame()
for i in df_list:
    df_=pd.read_csv(i)
    rr=pd.concat([rr, df_], ignore_index=True)
rr=rr.drop(["cohort","dt_id","animal_id","group_id"], axis=1).groupby(by=["min_15bin", "gander", "id_cohort"], as_index=False).mean()



# total orient index list
ori_list=[ disk_path+'\\CPT_TOT_manuscript\\data processing\\fig\\15_ori_index_LH_total\\female_total_ori_index_in_LH_15min_bin_ori_index_total_algorithm2.csv',
 disk_path+'\\CPT_TOT_manuscript\\data processing\\fig\\15_ori_index_LH_total\\male_total_ori_index_in_LH_15min_bin_ori_index_total_algorithm2.csv']

for i in ori_list:
    ori_df=pd.read_csv(i)
    rr_df=rr[rr["gander"]==ori_df["gander"][2]].copy()
    rr_df.to_csv(path+"\%s_response_rate.csv"%(ori_df["gander"][2]))
    
    if ori_df["gander"][2]=="male":
        p='royalblue'
        #title="male_total_ori_index_in_LH_15min_bin"
    else:
        p='coral'
        #title="female_total_ori_index_in_LH_15min_bin"
        
    plot=sns.pointplot(data=ori_df, x="min_15bin", y="ori_index_total_algorithm2", capsize=.15, errorbar=('ci', 68), color=p, errwidth=1)
    #plt.setp(plot.collections, sizes=[70])
    plot.set(xlabel="Bin")
    plot.set(ylabel="Total orientation index")
    new_labels = ["1", "2", "3", "4", "5", "6"]
    plot.set_xticklabels(new_labels)   
    plt.ylim(0.0, 1.0)
    plt.legend(bbox_to_anchor=(1.30, 1),borderaxespad=0)
    plot.spines['right'].set_visible(False)
    plot.spines['top'].set_visible(False)
    
    ax2=plot.twinx()
    bar=sns.barplot(data=rr_df, x="min_15bin", y="response_rate", capsize=.15, errorbar=('ci', 68), color=p, errwidth=1, ax=ax2)
    bar.set(ylabel="Response rate")
    new_labels = ["1", "2", "3", "4", "5", "6"]
    bar.set_xticklabels(new_labels)
    bar.set(xlabel="Bin")
    plt.ylim(0.0, 0.6)
    bar.spines['top'].set_visible(False)
    
    plt.savefig(path+"\%s_total_ori_index_response_rate.png"%(ori_df["gander"][2]), dpi=800, bbox_inches="tight")
    plt.close()


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
